In [3]:
import cv2
import numpy as np
import os
import mediapipe as mp
import math

In [4]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
holistic = mp_holistic.Holistic(min_detection_confidence = 0.8, min_tracking_confidence = 0.5)

In [5]:
IMAGESET_FOLDER = os.path.join('Imageset')
KEYPOINT_FOLDER = os.path.join('Extracted Hand Landmarks')
DATASET_FOLDER = os.path.join('Dataset')
actions = np.array(['I_word', 'hello', 'my', 'india'])
no_sequences = 120
no_frames = 30

In [6]:
def mediapipe_detection(image, holistic):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    result = holistic.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, result

In [7]:
def render_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [8]:
def get_true_coords(results):
    l_coords = np.zeros(shape = (21, 3), dtype = float)
    r_coords = np.zeros(shape = (21, 3), dtype = float)
    
#     l_shift_coords = np.zeros(shape = (21, 3), dtype = float)
#     r_shift_coords = np.zeros(shape = (21, 3), dtype = float)
    
    if results.left_hand_landmarks:
        for idxL, lh in enumerate(results.left_hand_landmarks.landmark):
            l_coords[idxL] = np.array([lh.x, lh.y, lh.z])
#         l_shift_coords = coord_shift(l_coords, 0)
    
    if results.right_hand_landmarks:
        for idxR, rh in enumerate(results.right_hand_landmarks.landmark):
            r_coords[idxR] = np.array([rh.x, rh.y, rh.z])
#         r_shift_coords = coord_shift(r_coords, 1)
    
    return l_coords, r_coords

In [9]:
def get_shifted_coords(coords_array, num):
    
    coords_zero = np.all((coords_array == 0))
    shifted_coords_array = np.zeros(shape = (21, 3), dtype = float)
    
    new_l_x = 8.24e-01
    new_l_y = 6.7e-01
    new_l_z = 2.25e-07
    
    new_r_x = 1.39e-01
    new_r_y = 6.7e-01
    new_r_z = 2.25e-07
    
    if num == 0:
        shifted_coords_array[0] = np.array([new_l_x, new_l_y, new_l_z])
        shift_factor_x = new_l_x - coords_array[0][0]
        shift_factor_y = new_l_y - coords_array[0][1]
        shift_factor_z = new_l_z - coords_array[0][2]
        for i in range(1, len(coords_array)):
            shifted_coords_array[i] = np.array([(coords_array[i][0] + shift_factor_x), (coords_array[i][1] + shift_factor_y), (coords_array[i][2] + shift_factor_z)])

    else:
        shifted_coords_array[0] = np.array([new_r_x, new_r_y, new_r_z])
        shift_factor_x = new_r_x - coords_array[0][0]
        shift_factor_y = new_r_y - coords_array[0][1]
        shift_factor_z = new_r_z - coords_array[0][2]
        for j in range(1, len(coords_array)):
            shifted_coords_array[j] = np.array([(coords_array[j][0] + shift_factor_x), (coords_array[j][1] + shift_factor_y), (coords_array[j][2] + shift_factor_z)])

    #print(shifted_coords_array)
    return shifted_coords_array

In [10]:
def hand_wrist_distance(coords_array):
    
    wrist_distance = np.zeros(shape = (21, 1), dtype = float)
    
    for num in range(len(coords_array)):
        wrist_distance[num] = (((coords_array[0][0] - coords_array[num][0])**2) + ((coords_array[0][1] - coords_array[num][1])**2) + ((coords_array[0][2] - coords_array[num][2])**2))**0.5
        
    return wrist_distance

In [11]:
def calculate_angles(results):
    
    #right_elbow, right_shoulder, left_shoulder, left_elbow
    four_joint_angles = np.zeros(shape = (4, 1), dtype = float)
    
    if results.pose_landmarks:
        # angle_joints = [right_elbow, right_shoulder, left_shoulder, left_elbow]
        angle_joints = [[16, 14, 12], [11, 12, 14], [13, 11, 12], [11, 13, 15]]
        
        for joint_index in range(len(angle_joints)):
            
            p1 = np.array([results.pose_landmarks.landmark[angle_joints[joint_index][0]].x, results.pose_landmarks.landmark[angle_joints[joint_index][0]].y])
            p2 = np.array([results.pose_landmarks.landmark[angle_joints[joint_index][1]].x, results.pose_landmarks.landmark[angle_joints[joint_index][1]].y])
            p3 = np.array([results.pose_landmarks.landmark[angle_joints[joint_index][2]].x, results.pose_landmarks.landmark[angle_joints[joint_index][2]].y])
            
            angle = math.degrees(math.atan2(p3[1] - p2[1], p3[0] - p2[0]) - math.atan2(p1[1] - p2[1], p1[0] - p2[0]))
            
            if angle < 0:
                angle = angle + 360
            
            rad_angle = angle * (np.pi / 180)
            
            normalised_angle = (angle / 360)
                
            four_joint_angles[joint_index] = angle
    
    return four_joint_angles

In [12]:
def nose_to_joint(results):
    
    nose_to_joints_distance = np.zeros(shape = (6, 1), dtype = float)
    
    if results.pose_landmarks:
        
        #right_wrist, right_elbow, right_shoulder, left_shoulder, left_elbow, left_wrist
        nose_to_joints = [16, 14, 12, 11, 13, 15]
        
        for distance_idx in range(len(nose_to_joints)):
            
            nose_to_joints_distance[distance_idx] = (((results.pose_landmarks.landmark[0].x - results.pose_landmarks.landmark[nose_to_joints[distance_idx]].x) ** 2) + ((results.pose_landmarks.landmark[0].y - results.pose_landmarks.landmark[nose_to_joints[distance_idx]].y) ** 2)) ** 0.5
    
    return nose_to_joints_distance

In [13]:
def extract_feature_points(results):
    
    l_true_coords = np.zeros(shape = (21, 3), dtype = float)
    r_true_coords = np.zeros(shape = (21, 3), dtype = float)
    
    l_shift_coords = np.zeros(shape = (21, 3), dtype = float)
    r_shift_coords = np.zeros(shape = (21, 3), dtype = float)
    
    l_h_wrist_distance = np.zeros(shape = (21, 1), dtype = float)
    r_h_wrist_distance = np.zeros(shape = (21, 1), dtype = float)
    
    nose_joint_distances = np.zeros(shape = (6, 1), dtype = float)
    
    final_nose_joint_distances = np.zeros(shape = (6, 1), dtype = float)
    
    four_joint_angles = np.zeros(shape = (4, 1), dtype = float)
    
    final_four_joint_angles = np.zeros(shape = (4, 1), dtype = float)
    
    if results:
        l_true_coords, r_true_coords = get_true_coords(results)
        
        four_joint_angles = calculate_angles(results)
        
        nose_joint_distances = nose_to_joint(results)
        
        if results.left_hand_landmarks:
            l_shift_coords = get_shifted_coords(l_true_coords, 0)
            l_h_wrist_distance = hand_wrist_distance(l_shift_coords)
            four_joint_angles[0] = 0
            four_joint_angles[1] = 0
            nose_joint_distances[0] = 0
            nose_joint_distances[1] = 0
            nose_joint_distances[2] = 0
#             final_nose_joint_distances = np.array([0, 0, 0, nose_joint_distances[3], nose_joint_distances[4], nose_joint_distances[5]])
#             final_four_joint_angles = np.array([0, 0, four_joint_angles[2], four_joint_angles[3]], dtype = float)
        
        if results.right_hand_landmarks:
            r_shift_coords = get_shifted_coords(r_true_coords, 1)
            r_h_wrist_distance = hand_wrist_distance(r_shift_coords)
            four_joint_angles[2] = 0
            four_joint_angles[3] = 0
            nose_joint_distances[3] = 0
            nose_joint_distances[4] = 0
            nose_joint_distances[5] = 0
#             final_nose_joint_distances = np.array([nose_joint_distances[0], nose_joint_distances[1], nose_joint_distances[2], 0, 0, 0])
#             final_four_joint_angles = np.array([four_joint_angles[0], four_joint_angles[1], 0, 0], dtype = float)
            
        return np.concatenate((l_shift_coords.flatten(), r_shift_coords.flatten(), l_h_wrist_distance.flatten(), r_h_wrist_distance.flatten(), four_joint_angles.flatten(), nose_joint_distances.flatten())).flatten()
#     return np.column_stack((l_shift_coords, r_shift_coords, l_h_wrist_distance, r_h_wrist_distance, l_r_hand_distance)).flatten()

In [14]:
for action in range(len(actions)):
    for seqNo in range(no_sequences):
        for frameNo in range(no_frames):
            source_image_path = os.path.join(IMAGESET_FOLDER, actions[action], str(seqNo))
            image = cv2.imread(os.path.join(source_image_path, str(frameNo) + '.jpg'))
            
            image = cv2.flip(image, 1)
            
            image, results = mediapipe_detection(image, holistic)
        
            render_landmarks(image, results)
            
            frame_features = extract_feature_points(results)
            
            saved_image_path = os.path.join(KEYPOINT_FOLDER, actions[action], str(seqNo))
            
            image = cv2.flip(image, 1)
            try:
                os.makedirs(saved_image_path)
            except:
                pass
            
            cv2.imwrite(os.path.join(saved_image_path, str(frameNo) + '.jpg'), image)
            
            npy_path = os.path.join(DATASET_FOLDER, actions[action], str(seqNo))
            
            try:
                os.makedirs(npy_path)
            except:
                pass
            np.save(os.path.join(npy_path, str(frameNo)), frame_features)